<a href="https://colab.research.google.com/github/corentinBlanchard/chatbot-IA/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports pour recuperation de fichiers a partir de google drive ( ou la DB est stockee)
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
#Importation des donnees sur la conversation concernant l'IA
downloaded = drive.CreateFile({'id':'1asz0LQpmjqAfx7_rc6WxkOeYOS848F2S'})
downloaded.GetContentFile('ia.yml') 

In [4]:
import yaml
file = open('ia.yml')
docs = yaml.safe_load(file)
print(docs['conversations'][0])

['What is AI?', 'Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.']


In [5]:
#preprocessing : nous decoupons notre BDD en mots en supprimant la ponctuation afin de créer un dictionnaire
words = []
#recuperation des mots 
for sentences in docs['conversations'] :
  question = sentences[0].split()
  #answer = sentences[1].split()
  for word in question:
    words.append(word.lower())
  #for word in answer :
  #  words.append(word.lower())
print(words[:50])

['what', 'is', 'ai?', 'what', 'is', 'ai?', 'are', 'you', 'sentient?', 'are', 'you', 'sentient?', 'are', 'you', 'sentient?', 'are', 'you', 'sapient?', 'are', 'you', 'sapient?', 'are', 'you', 'sapient?', 'are', 'you', 'sapient?', 'what', 'language', 'are', 'you', 'written', 'in?', 'what', 'language', 'are', 'you', 'written', 'in?', 'you', 'sound', 'like', 'data', 'you', 'sound', 'like', 'data', 'you', 'are', 'an']


In [6]:
#suppression de la ponctuation
import string
import re
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
stripped = [re_punc.sub('', w) for w in words]
print(stripped[:50])

['what', 'is', 'ai', 'what', 'is', 'ai', 'are', 'you', 'sentient', 'are', 'you', 'sentient', 'are', 'you', 'sentient', 'are', 'you', 'sapient', 'are', 'you', 'sapient', 'are', 'you', 'sapient', 'are', 'you', 'sapient', 'what', 'language', 'are', 'you', 'written', 'in', 'what', 'language', 'are', 'you', 'written', 'in', 'you', 'sound', 'like', 'data', 'you', 'sound', 'like', 'data', 'you', 'are', 'an']


In [7]:
#Importation des stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [9]:
#Suppression des stopwords dans notre BDD

import string
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
words = [w for w in stripped if not w in stop_words]
print(words[:100])

['ai', 'ai', 'sentient', 'sentient', 'sentient', 'sapient', 'sapient', 'sapient', 'sapient', 'language', 'written', 'language', 'written', 'sound', 'like', 'data', 'sound', 'like', 'data', 'artificial', 'linguistic', 'entity', 'artificial', 'linguistic', 'entity', 'immortal', 'immortal', 'immortal', 'making', 'sense', 'making', 'sense', 'making', 'sense', 'making', 'sense', 'making', 'sense', 'immortal', 'immortal', 'immortal', 'make', 'sense', 'clone', 'clone', 'move', 'move', 'bend', 'bend', 'robots', 'laugh', 'robots', 'die', 'robots', 'robots', 'stupid', 'robots', 'allowed', 'lie', 'robots', 'allowed', 'lie', 'robots', 'allowed', 'lie', 'robotics', 'computer', 'computer', 'walk', 'walk', 'fight', 'die', 'die', 'die', 'die', 'chat', 'robot', 'chat', 'robot', 'chat', 'bot', 'chatterbox', 'chatterbox', 'motormouth', 'ratchet', 'jaw', 'robot', 'body', 'robot', 'body', 'business', 'business', 'favorite', 'programming', 'language', 'favorite', 'programming', 'language', 'favorite', 'hobb

In [10]:
#Stemming pour réduction des mots à leur base
from nltk.stem.porter import PorterStemmer
# stemming of words
porter = PorterStemmer()
words = [porter.stem(word) for word in words]
print(words[:100])

['ai', 'ai', 'sentient', 'sentient', 'sentient', 'sapient', 'sapient', 'sapient', 'sapient', 'languag', 'written', 'languag', 'written', 'sound', 'like', 'data', 'sound', 'like', 'data', 'artifici', 'linguist', 'entiti', 'artifici', 'linguist', 'entiti', 'immort', 'immort', 'immort', 'make', 'sens', 'make', 'sens', 'make', 'sens', 'make', 'sens', 'make', 'sens', 'immort', 'immort', 'immort', 'make', 'sens', 'clone', 'clone', 'move', 'move', 'bend', 'bend', 'robot', 'laugh', 'robot', 'die', 'robot', 'robot', 'stupid', 'robot', 'allow', 'lie', 'robot', 'allow', 'lie', 'robot', 'allow', 'lie', 'robot', 'comput', 'comput', 'walk', 'walk', 'fight', 'die', 'die', 'die', 'die', 'chat', 'robot', 'chat', 'robot', 'chat', 'bot', 'chatterbox', 'chatterbox', 'motormouth', 'ratchet', 'jaw', 'robot', 'bodi', 'robot', 'bodi', 'busi', 'busi', 'favorit', 'program', 'languag', 'favorit', 'program', 'languag', 'favorit', 'hobbi']


In [11]:
# Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
# create the transform
# vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(words)
print("Bag of words :")
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(words)
# summarize encoded vector
print("vector shape : ")
print(vector.shape)
print("vector size : ")
print(vector.size)

Bag of words :
{'ai': 0, 'sentient': 45, 'sapient': 43, 'languag': 28, 'written': 56, 'sound': 48, 'like': 31, 'data': 14, 'artifici': 2, 'linguist': 32, 'entiti': 16, 'immort': 25, 'make': 33, 'sens': 44, 'clone': 10, 'move': 37, 'bend': 3, 'robot': 42, 'laugh': 29, 'die': 15, 'stupid': 49, 'allow': 1, 'lie': 30, 'comput': 11, 'walk': 54, 'fight': 19, 'chat': 8, 'bot': 5, 'chatterbox': 9, 'motormouth': 36, 'ratchet': 41, 'jaw': 26, 'bodi': 4, 'busi': 7, 'favorit': 18, 'program': 40, 'hobbi': 22, 'idea': 24, 'shoe': 46, 'size': 47, 'oper': 38, 'system': 50, 'type': 52, 'kind': 27, 'hardwar': 21, 'hope': 23, 'want': 55, 'cramp': 13, 'true': 51, 'ever': 17, 'mate': 35, 'go': 20, 'breath': 6, 'control': 12, 'malfunct': 34, 'use': 53, 'product': 39}
vector shape : 
(163, 57)
vector size : 
163


In [45]:
# Create table with questions and preprocessing them

questions_text = []
questions_bow = []


for sentences in docs['conversations'] :
  question = sentences[0].split()
  table = []
  merged_data = []
  
  for word in question:
    table.append(word.lower())
  
  table = [re_punc.sub('', w) for w in table]
  table = [w for w in table if not w in stop_words]
  table = [porter.stem(word) for word in table]
  merged_data.append(' '.join(table))

  vector = vectorizer.transform(merged_data)
  #print(vector.toarray())

  questions_text.append(merged_data)
  questions_bow.append(vector)

print(questions_text)
print(questions_bow[12].toarray())


[['ai'], ['ai'], ['sentient'], ['sentient'], ['sentient'], ['sapient'], ['sapient'], ['sapient'], ['sapient'], ['languag written'], ['languag written'], ['sound like data'], ['sound like data'], ['artifici linguist entiti'], ['artifici linguist entiti'], ['immort'], ['immort'], ['immort'], ['make sens'], ['make sens'], ['make sens'], ['make sens'], ['make sens'], ['immort'], ['immort'], ['immort'], ['make sens'], ['clone'], ['clone'], ['move'], ['move'], ['bend'], ['bend'], ['robot laugh'], ['robot die'], ['robot'], ['robot stupid'], ['robot allow lie'], ['robot allow lie'], ['robot allow lie'], ['robot'], ['comput'], ['comput'], ['walk'], ['walk'], ['fight'], ['die'], ['die'], ['die'], ['die'], ['chat robot'], ['chat robot'], ['chat bot'], ['chatterbox'], ['chatterbox'], ['motormouth'], ['ratchet jaw'], ['robot bodi'], ['robot bodi'], ['busi'], ['busi'], ['favorit program languag'], ['favorit program languag'], ['favorit hobbi'], ['idea'], ['shoe size'], ['like robot'], ['like robot']